<a href="https://colab.research.google.com/github/giovannioliveira/knative-serving/blob/development/notebooks/EDA_trace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gc
import time
from datetime import datetime
import urllib3
from itertools import permutations
import pandas as pd
from pandas import MultiIndex
import numpy as np
from numpy import vectorize
from numpy.random import SeedSequence

EXECUTION_NAME = datetime.now().replace(second=0,microsecond=0).isoformat()
CHECKPOINT_PATH = '/content/drive/Shareddrives/faas/data/checkpoints/';
EXECUTION_DURATION_S = 14*24*60*60

n_samples = 300; max_int = 1000000000; min_int = -1000000000; num_base = 10;
TRUE_RAND_GEN_URL = f'https://www.random.org/integers/?\
num={"n_samples"}&format=plain&min={min_int}&max={max_int}\
&col=1&base={num_base}&md=new'; NS_TO_S = 1000000000;
http = urllib3.PoolManager()

def get_true_random(n_samples: int = n_samples, normalize: bool = True) -> list[float]:
  resp = urllib3.request('GET',
          TRUE_RAND_GEN_URL.replace('n_samples',str(n_samples)))
  resp.drain_conn()
  return list(map((lambda x: (int(x)/2e9 + .5) if normalize else int(x)),
             str(resp.data, encoding='utf-8').split('\n')[:-1]))
seedz = get_true_random(n_samples, False)
print('random seeds: ',seedz)
RANDOM_GEN = np.random.default_rng(seed=SeedSequence(entropy=list(map(abs,seedz))))

random seeds:  [499874867, -399169840, 177032723, 691625829, -266479958, -960913958, -32596412, -200805874, 228556342, 783850822, 450361685, -688634953, 123424919, 207315672, 374023088, -307423341, 985469698, -304557797, 76541056, 106567183, -512670384, 406086624, 158379123, 814386309, 641224540, -203765409, -947594972, -930459228, 650151545, -641172616, -142656268, 193082647, 259626386, -5616437, 908546738, 699170460, 340149114, 818924646, -406506503, -92883816, -855889714, -895831008, 225850965, -983632279, -307438338, -29140895, -648829087, -777132790, 599169025, -568436466, 916011552, -192565923, -809075929, 286276888, -603543479, 602955895, 4563482, 568752365, 438089725, 944538088, -684507925, -159283184, 911320351, 147988967, 113309, 285321925, 221529381, -164024258, 400616051, -599693216, -555965150, -841793816, -649243359, -408297242, 889980090, -245792046, -181423170, -909954656, -155519527, 571752418, 626817923, -147972326, 993213109, 350392146, -392335689, -623448509, -28138

In [ ]:
_df : (pd.DataFrame | None) = pd.read_csv(
    "/content/drive/Shareddrives/faas/data/AzureFunctionsInvocationTraceForTwoWeeksJan2021.txt")
_df

,app,func,end_timestamp,duration
0,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,e3cdb48830f66eb8689cc0223514569a69812b77e6611e...,7.949090e-02,0.078
1,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,337cd24a7d5fd5c92460faee4ebe6a186a0eb322bd17b7...,5.715786e+01,57.154
2,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,48cc770d590d3c5a7691b3b4e9302f82ec3be5ddc2a037...,5.913048e+01,59.125
3,f274d71de386ccc77e4ca74766dbc485461c3053059d47...,3d2aee54a133509f16fb636d74128c2adcfcac71c6dcef...,6.252541e+00,6.236
4,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,68bbfd828223a505d7917339f4656c5f33ff93225cdb9d...,6.682396e-02,0.050
...,...,...,...,...
1980946,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209597e+06,0.001
1980947,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209598e+06,0.001
1980948,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001
1980949,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001


In [ ]:
app_idx = _df["app"].unique()
RANDOM_GEN.shuffle(app_idx)
app_idx_enum = dict([x, idx] for idx, x
                    in enumerate(app_idx))
app_id_map = lambda app_id: app_idx_enum[app_id]
_df["app_id"] = _df["app"].apply(func=app_id_map)
func_idx = _df["func"].unique()
RANDOM_GEN.shuffle(func_idx)
func_idx_enum = dict([x, idx] for idx, x
                    in enumerate(func_idx))
func_id_map = lambda func_idx: func_idx_enum[func_idx]
_df["func_id"] = _df["func"].apply(func=func_id_map)
_df["end_ns"] = (NS_TO_S * _df["end_timestamp"]).astype(int)
_df["dur_ns"] = (NS_TO_S * _df["duration"]).astype(int)
_df['beg_ns'] = _df['end_ns'] - _df["dur_ns"]
_df["ts"] = _df['beg_ns']
_df.sort_values(by=["ts"],ignore_index=True, ascending=True, inplace=True)
_df["req_id"] = _df.index
_df["event_type"] = 1
_df

,app,func,end_timestamp,duration,app_id,func_id,end_ns,dur_ns,beg_ns,ts,req_id,event_type
0,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,e3cdb48830f66eb8689cc0223514569a69812b77e6611e...,7.949090e-02,0.078,56,310,79490900,78000000,1490900,1490900,0,1
1,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,337cd24a7d5fd5c92460faee4ebe6a186a0eb322bd17b7...,5.715786e+01,57.154,15,37,57157860040,57154000000,3860040,3860040,1,1
2,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,48cc770d590d3c5a7691b3b4e9302f82ec3be5ddc2a037...,5.913048e+01,59.125,15,125,59130477905,59125000000,5477905,5477905,2,1
3,f274d71de386ccc77e4ca74766dbc485461c3053059d47...,3d2aee54a133509f16fb636d74128c2adcfcac71c6dcef...,6.252541e+00,6.236,71,212,6252541065,6236000000,16541065,16541065,3,1
4,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,68bbfd828223a505d7917339f4656c5f33ff93225cdb9d...,6.682396e-02,0.050,56,316,66823959,50000000,16823959,16823959,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1980946,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209597e+06,0.001,115,80,1209596701016903,1000000,1209596700016903,1209596700016903,1980946,1
1980947,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209598e+06,0.001,115,80,1209598191409111,1000000,1209598190409111,1209598190409111,1980947,1
1980948,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001,115,80,1209598561518907,1000000,1209598560518907,1209598560518907,1980948,1
1980949,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001,115,80,1209598628381967,1000000,1209598627381967,1209598627381967,1980949,1


In [ ]:
_df.drop(columns=["app","func","end_timestamp","duration"], inplace=True)
_df.to_csv(CHECKPOINT_PATH+"-FAASCHEDULE.csv.bz2",
           columns=["ts","dur_ns","func_id","app_id"], index=True,
           index_label="id")

In [ ]:
_df_dept = _df.sort_values(by="end_ns", ignore_index=True, ascending=True)
_df_dept["ts"] = _df["end_ns"]
_df_dept["event_type"] = 0
_df_all = pd.concat([_df, _df_dept])
_df_all.sort_values(by=["ts","event_type"],ignore_index = True, ascending=[True,False], inplace = True)

In [ ]:
DF = (_df_all.copy(deep=True))[["ts", "event_type", "req_id", "func_id", "app_id", "beg_ns", "end_ns", "dur_ns" ]]
del _df, _df_dept, _df_all
gc.collect()

13

In [ ]:
DF["group"] = (DF["ts"]*1e-9).astype(int)
DF.to_csv(CHECKPOINT_PATH+EXECUTION_NAME+'-EVENTS.csv.bz2',index=True, index_label="id")
DF  # this is a checkpoint for event preprocessing, next we'll augment data with some metrics

,ts,event_type,req_id,func_id,app_id,beg_ns,end_ns,dur_ns,group
0,1490900,1,0,310,56,1490900,79490900,78000000,0
1,3860040,1,1,37,15,3860040,57157860040,57154000000,0
2,5477905,1,2,125,15,5477905,59130477905,59125000000,0
3,16541065,1,3,212,71,16541065,6252541065,6236000000,0
4,16823959,1,4,316,56,16823959,66823959,50000000,0
...,...,...,...,...,...,...,...,...,...
3961897,1209598561518907,0,1980948,80,115,1209598560518907,1209598561518907,1000000,1209598
3961898,1209598627381967,1,1980949,80,115,1209598627381967,1209598628381967,1000000,1209598
3961899,1209598628381967,0,1980949,80,115,1209598627381967,1209598628381967,1000000,1209598
3961900,1209599693463968,1,1980950,80,115,1209599693463968,1209599694463968,1000000,1209599


In [4]:
try:
    DF
    print('DF exists')
except(NameError):
    BACKUP_EVENTS_FILE = '/content/drive/Shareddrives/faas/data/checkpoints/2024-04-10T23:59:00-EVENTS.csv.bz2'
    DF = pd.read_csv(BACKUP_EVENTS_FILE)

In [5]:
DF

,id,ts,event_type,req_id,func_id,app_id,beg_ns,end_ns,dur_ns,group
0,0,1490900,1,0,236,90,1490900,79490900,78000000,0
1,1,3860040,1,1,165,110,3860040,57157860040,57154000000,0
2,2,5477905,1,2,408,110,5477905,59130477905,59125000000,0
3,3,16541065,1,3,398,88,16541065,6252541065,6236000000,0
4,4,16823959,1,4,189,90,16823959,66823959,50000000,0
...,...,...,...,...,...,...,...,...,...,...
3961897,3961897,1209598561518907,0,1980948,25,36,1209598560518907,1209598561518907,1000000,1209598
3961898,3961898,1209598627381967,1,1980949,25,36,1209598627381967,1209598628381967,1000000,1209598
3961899,3961899,1209598628381967,0,1980949,25,36,1209598627381967,1209598628381967,1000000,1209598
3961900,3961900,1209599693463968,1,1980950,25,36,1209599693463968,1209599694463968,1000000,1209599


Features to augment dataset:
 *  Event-based metrics
    *  last_event_ns
    *  last_arv_ns
    *  last_dpt_ns
    *  last_func_event_ns
    *  last_func_arv_ns
    *  last_func_dpt_ns
    *  last_app_event_ns
    *  last_app_arv_ns
    *  last_app_id_arr
    *  last_app_dpt_ns  
    *  last_app_id_dpt
    *  cur_req_count
    *  cur_app_count
    *  cur_func_count
    *  cur_func_req_count
      * or, OHE? app_func_req_count
    *  cur_app_req_count

In [6]:
DF_ = dict()
DF_['time'] = DF.groupby(by="group")
DF_['func'] = DF.groupby(by="func_id")
DF_['app'] = DF.groupby(by="app_id")
DF_['event_type'] = DF.groupby(by="event_type")
DF_['dur'] = DF.groupby(pd.cut(DF["dur_ns"],(0, 3.100000e+07, 3.810000e+08,
                                         1.620000e+09, 1.441600e+10, 7.219999e+10,
                                         2.536556e+11, 4.660390e+11, 5.786200e+11)))

In [ ]:
DF['last_event_ns'] = np.append([DF["ts"].iloc[0]],
                                DF["ts"].iloc[1:].values - DF["ts"].iloc[:-1].values)
DF['last_arv_ns'] = vectorize(DF)

In [41]:
DF.iloc[0]

id                   0
ts             1490900
event_type           1
req_id               0
func_id            236
app_id              90
beg_ns         1490900
end_ns        79490900
dur_ns        78000000
group                0
Name: 0, dtype: int64

In [ ]:
set

In [26]:
def get_backward_index(base_indices: np.array, raw_indices: np.array, method: str = 'ffill', force_prev: bool = True) -> pd.Index:
  if force_prev:
    def _iiiiiii(x:int, y: set) -> int:
      return x-1 if (x in y) else x
    indices = vectorize(pyfunc=_iiiiiii)
  return base_indices.get_indexer(target=indices(raw_indices.values,base_indices), method=method)

get_backward_index(DF_["event_type"].get_group(0).index,DF.index)


TypeError: argument of type 'numpy.int64' is not iterable

In [39]:
def m(a,b,c):
  return a*b*c

mv = vectorize(m)
mv([2,3,5],7,19)

array([266, 399, 665])

In [ ]:
DF_['event_type'].get_group(0)

,id,ts,event_type,req_id,func_id,app_id,beg_ns,end_ns,dur_ns,group
16,16,66823959,0,19,183,47,175120000,795120000,620000000,0
18,18,79490900,0,4,189,90,16823959,66823959,50000000,0
19,19,99991083,0,11,193,93,23631021,1126631021,1103000000,0
24,24,200403928,0,14,364,104,56239952,11117239952,11061000000,0
26,26,795120000,0,29,314,47,15345080984,17307080984,1962000000,0
...,...,...,...,...,...,...,...,...,...,...
3961893,3961893,1209596701016903,0,1980946,25,36,1209596700016903,1209596701016903,1000000,1209596
3961895,3961895,1209598191409111,0,1980947,25,36,1209598190409111,1209598191409111,1000000,1209598
3961897,3961897,1209598561518907,0,1980948,25,36,1209598560518907,1209598561518907,1000000,1209598
3961899,3961899,1209598628381967,0,1980949,25,36,1209598627381967,1209598628381967,1000000,1209598


.<BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR>






















































.







In [ ]:
timeseriesBuckets = np.linspace(0,EXECUTION_DURATION_S,EXECUTION_DURATION_S)
nTimeseriesBuckets = len(timeseriesBuckets)
data = [None, ]*5


In [ ]:
DF.describe(percentiles=[.5,.77,.9,.95,.99,.999,.9999])

In [ ]:
def _timeseries_count_prop_unique(timeseriesBucket, prop):
  return len(DF_['time'].get_group(int(timeseriesBucket))[prop].unique())
timeseries_count_prop_unique = np.vectorize(_timeseries_count_prop_unique)
timeseries_count_prop_unique(timeseriesBuckets, 'req_id')

KeyError: 3

0

In [ ]:
DF.query("900 < group < 960")

,id,ts,event_type,req_id,func_id,app_id,beg_ns,end_ns,dur_ns,group
325,325,901011060979,1,171,5,93,901011060979,902708060979,1697000000,901
326,326,902111104965,0,135,314,47,628451958932,928286958932,299835000000,902
327,327,902126868963,0,122,314,47,628433652046,938280652046,309847000000,902
328,328,902348504066,0,126,314,47,628440817934,935159817934,306719000000,902
329,329,902439243078,0,131,314,47,628444309938,931306309938,302862000000,902
330,330,902708060979,0,174,93,47,960000836942,960127836942,127000000,902
331,331,906231925964,0,163,10,93,900009243078,902439243078,2430000000,906
332,332,906286443105,1,172,21,109,906286443105,948245443105,41959000000,906
333,333,909059370994,0,121,314,47,628292790048,939899790048,311607000000,909
334,334,910193922996,0,123,314,47,628435586894,938646586894,310211000000,910
